# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

### a PromptLogs object will hold all the individual individual parser objects, which hold the response from a single exchange

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

# a MessageParser object will process the message_list and make an acceptable messages object
message_parser = MessageParser(message_list)
messages = message_parser.parse()

# Make the openai call and get the response
# todo: I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


A response parser will process the response

In [7]:
response_parser = ResponseParser(response=response)

See a pandas dataframe of the logged response

In [8]:
response_parser.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-756ZeJhW8dAUz4O3ewkvngOmb50Ln,1.681451e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;generates&nbsp;Fibonacci&nbsp;sequence&nbsp;up&nbsp;to&nbsp;'n'&nbsp;numbers.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;0,&nbsp;1<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;a&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;print(a)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;b,&nbsp;a+b<br>#Example<br>fibonacci(10)<br>def&nbsp;zero_pad(s):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;pads&nbsp;a&nbsp;string&nbsp;'s'&nbsp;with&nbsp;zeros&nbsp;<br>&nbsp;&nbsp;&nbsp;&nbsp;up&nbsp;to&nbsp;a&nbsp;length&nbsp;of&nbsp;10&nbsp;characters.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;s.zfill(10)<br>#Example<br>from&nbsp;string_functions&nbsp;import&nbsp;zero_pad<br>my_string&nbsp;=&nbsp;""42""<br>print(zero_pad(my_string))<br>","[\ndef fibonacci(n):\n """"""\n This function generates Fibonacci sequence up to 'n' numbers.\n """"""\n a, b = 0, 1\n while a < n:\n print(a)\n a, b = b, a+b\n, \n#Example\nfibonacci(10)\n, \ndef zero_pad(s):\n """"""\n This function pads a string 's' with zeros \n up to a length of 10 characters.\n """"""\n return s.zfill(10)\n, \n#Example\nfrom string_functions import zero_pad\nmy_string = ""42""\nprint(zero_pad(my_string))\n]","Here is the python code for fibonacci sequence:\n\n```python\ndef fibonacci(n):\n """"""\n This function generates Fibonacci sequence up to 'n' numbers.\n """"""\n a, b = 0, 1\n while a < n:\n print(a)\n a, b = b, a+b\n```\n\nTo use this function, simply call the 'fibonacci()`' function and pass the integer value of n.\n\n```python\n#Example\nfibonacci(10)\n```\n\nOutput:\n```\n0\n1\n1\n2\n3\n5\n8\n```\n\nAnd, here is the function to zero pad a string to 10 characters in a separate file named `string_functions.py`.\n\n`string_functions.py`\n\n```python\ndef zero_pad(s):\n """"""\n This function pads a string 's' with zeros \n up to a length of 10 characters.\n """"""\n return s.zfill(10)\n```\n\nYou can import and use the 'zero_pad()' function in any other file as below:\n\n```python\n#Example\nfrom string_functions import zero_pad\nmy_string = ""42""\nprint(zero_pad(my_string))\n```\n\nOutput:\n```\n0000000042\n```"


In [9]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
response_parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-756ZeJhW8dAUz4O3ewkvngOmb50Ln,1681451266.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): """""" This function generates Fibonacci sequence up to 'n' numbers. """""" a, b = 0, 1 while a < n: print(a) a, b = b, a+b#Examplefibonacci(10)def zero_pad(s): """""" This function pads a string 's' with zeros up to a length of 10 characters. """""" return s.zfill(10)#Examplefrom string_functions import zero_padmy_string = ""42""print(zero_pad(my_string))"


In [10]:
# this is a PromptLogs object again. We'll add the response_parser to the logs
logs.append(messages, response_parser)
logs.styled_log

,id,created,model,usage,prompts,all_code,content
sid,,,,,,,
a1,chatcmpl-756ZeJhW8dAUz4O3ewkvngOmb50Ln,1681451266.000000,gpt-3.5-turbo-0301,66.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]","def fibonacci(n): """""" This function generates Fibonacci sequence up to 'n' numbers. """""" a, b = 0, 1 while a < n: print(a) a, b = b, a+b#Examplefibonacci(10)def zero_pad(s): """""" This function pads a string 's' with zeros up to a length of 10 characters. """""" return s.zfill(10)#Examplefrom string_functions import zero_padmy_string = ""42""print(zero_pad(my_string))","Here is the python code for fibonacci sequence: ```python def fibonacci(n): """""" This function generates Fibonacci sequence up to 'n' numbers. """""" a, b = 0, 1 while a < n: print(a) a, b = b, a+b ``` To use this function, simply call the 'fibonacci()`' function and pass the integer value of n. ```python #Example fibonacci(10) ``` Output: ``` 0 1 1 2 3 5 8 ``` And, here is the function to zero pad a string to 10 characters in a separate file named `string_functions.py`. `string_functions.py` ```python def zero_pad(s): """""" This function pads a string 's' with zeros up to a length of 10 characters. """""" return s.zfill(10) ``` You can import and use the 'zero_pad()' function in any other file as below: ```python #Example from string_functions import zero_pad my_string = ""42"" print(zero_pad(my_string)) ``` Output: ``` 0000000042 ```"


## And we have another exchange

In [11]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(messages, response_parser)

In [12]:
logs.df

,id,created,model,usage,all_code,code_blocks,content,sid,prompts
0,chatcmpl-756ZeJhW8dAUz4O3ewkvngOmb50Ln,1.681451e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;generates&nbsp;Fibonacci&nbsp;sequence&nbsp;up&nbsp;to&nbsp;'n'&nbsp;numbers.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;0,&nbsp;1<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;a&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;print(a)<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;a,&nbsp;b&nbsp;=&nbsp;b,&nbsp;a+b<br>#Example<br>fibonacci(10)<br>def&nbsp;zero_pad(s):<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;This&nbsp;function&nbsp;pads&nbsp;a&nbsp;string&nbsp;'s'&nbsp;with&nbsp;zeros&nbsp;<br>&nbsp;&nbsp;&nbsp;&nbsp;up&nbsp;to&nbsp;a&nbsp;length&nbsp;of&nbsp;10&nbsp;characters.<br>&nbsp;&nbsp;&nbsp;&nbsp;""""""<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;s.zfill(10)<br>#Example<br>from&nbsp;string_functions&nbsp;import&nbsp;zero_pad<br>my_string&nbsp;=&nbsp;""42""<br>print(zero_pad(my_string))<br>","[\ndef fibonacci(n):\n """"""\n This function generates Fibonacci sequence up to 'n' numbers.\n """"""\n a, b = 0, 1\n while a < n:\n print(a)\n a, b = b, a+b\n, \n#Example\nfibonacci(10)\n, \ndef zero_pad(s):\n """"""\n This function pads a string 's' with zeros \n up to a length of 10 characters.\n """"""\n return s.zfill(10)\n, \n#Example\nfrom string_functions import zero_pad\nmy_string = ""42""\nprint(zero_pad(my_string))\n]","Here is the python code for fibonacci sequence:\n\n```python\ndef fibonacci(n):\n """"""\n This function generates Fibonacci sequence up to 'n' numbers.\n """"""\n a, b = 0, 1\n while a < n:\n print(a)\n a, b = b, a+b\n```\n\nTo use this function, simply call the 'fibonacci()`' function and pass the integer value of n.\n\n```python\n#Example\nfibonacci(10)\n```\n\nOutput:\n```\n0\n1\n1\n2\n3\n5\n8\n```\n\nAnd, here is the function to zero pad a string to 10 characters in a separate file named `string_functions.py`.\n\n`string_functions.py`\n\n```python\ndef zero_pad(s):\n """"""\n This function pads a string 's' with zeros \n up to a length of 10 characters.\n """"""\n return s.zfill(10)\n```\n\nYou can import and use the 'zero_pad()' function in any other file as below:\n\n```python\n#Example\nfrom string_functions import zero_pad\nmy_string = ""42""\nprint(zero_pad(my_string))\n```\n\nOutput:\n```\n0000000042\n```",a1,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]"
0,chatcmpl-756ZpYdXzk1tFuhiXPIwt84HcY2Lx,1.681451e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;get_list_of_numbers():<br>&nbsp;&nbsp;&nbsp;&nbsp;numbers&nbsp;=&nbsp;list(range(1,11))<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;numbers<br>","[\ndef get_list_of_numbers():\n numbers = list(range(1,11))\n return numbers\n]","Below is the Python function that returns numbers 1-10 in a list:\n\n```python\ndef get_list_of_numbers():\n numbers = list(range(1,11))\n return numbers\n```\nYou can save it in a file named `number_list.py` for future reuse. It can be later imported in any Python client file and run by simply invoking the `get_list_of_numbers()` function.",a2,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]"


### And print the styled logs with the code from two separate prompt exchanges

In [13]:
logs.styled_log

,id,created,model,usage,prompts,all_code,content
sid,,,,,,,
a1,chatcmpl-756ZeJhW8dAUz4O3ewkvngOmb50Ln,1681451266.000000,gpt-3.5-turbo-0301,66.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]","def fibonacci(n): """""" This function generates Fibonacci sequence up to 'n' numbers. """""" a, b = 0, 1 while a < n: print(a) a, b = b, a+b#Examplefibonacci(10)def zero_pad(s): """""" This function pads a string 's' with zeros up to a length of 10 characters. """""" return s.zfill(10)#Examplefrom string_functions import zero_padmy_string = ""42""print(zero_pad(my_string))","Here is the python code for fibonacci sequence: ```python def fibonacci(n): """""" This function generates Fibonacci sequence up to 'n' numbers. """""" a, b = 0, 1 while a < n: print(a) a, b = b, a+b ``` To use this function, simply call the 'fibonacci()`' function and pass the integer value of n. ```python #Example fibonacci(10) ``` Output: ``` 0 1 1 2 3 5 8 ``` And, here is the function to zero pad a string to 10 characters in a separate file named `string_functions.py`. `string_functions.py` ```python def zero_pad(s): """""" This function pads a string 's' with zeros up to a length of 10 characters. """""" return s.zfill(10) ``` You can import and use the 'zero_pad()' function in any other file as below: ```python #Example from string_functions import zero_pad my_string = ""42"" print(zero_pad(my_string)) ``` Output: ``` 0000000042 ```"
a2,chatcmpl-756ZpYdXzk1tFuhiXPIwt84HcY2Lx,1681451277.000000,gpt-3.5-turbo-0301,60.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]","def get_list_of_numbers(): numbers = list(range(1,11)) return numbers","Below is the Python function that returns numbers 1-10 in a list: ```python def get_list_of_numbers(): numbers = list(range(1,11)) return numbers ``` You can save it in a file named `number_list.py` for future reuse. It can be later imported in any Python client file and run by simply invoking the `get_list_of_numbers()` function."
